In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

# Census API key
from courtCensusKey import censusKey

# Import Data

In [2]:
#Import Data
#from https://simplemaps.com/data/us-zips

dataMain = pd.read_csv("Resources/uszips.csv")
dataMain

zip       lat        lng         city state_id   state_name  zcta  \
0        601  18.18004  -66.75218     Adjuntas       PR  Puerto Rico  True   
1        602  18.36073  -67.17517       Aguada       PR  Puerto Rico  True   
2        603  18.45439  -67.12202    Aguadilla       PR  Puerto Rico  True   
3        606  18.16724  -66.93828      Maricao       PR  Puerto Rico  True   
4        610  18.29032  -67.12243       Anasco       PR  Puerto Rico  True   
...      ...       ...        ...          ...      ...          ...   ...   
33092  99923  55.97796 -130.03671        Hyder       AK       Alaska  True   
33093  99925  55.55767 -132.97627      Klawock       AK       Alaska  True   
33094  99926  55.12617 -131.48928   Metlakatla       AK       Alaska  True   
33095  99927  56.25100 -133.37571  Point Baker       AK       Alaska  True   
33096  99929  56.36950 -131.93648     Wrangell       AK       Alaska  True   

       parent_zcta  population  density  county_fips            county_name  \
0              NaN       17242    111.4        72001               Adjuntas   
1              NaN       38442    523.5        72003                 Aguada   
2              NaN       48814    667.9        72005              Aguadilla   
3              NaN        6437     60.4        72093                Maricao   
4              NaN       27073    312.0        72011                 Añasco   
...            ...         ...      ...          ...                    ...   
33092          NaN          15      2.1         2198  Prince of Wales-Hyder   
33093          NaN         927      5.7         2198  Prince of Wales-Hyder   
33094          NaN        1635      4.2         2198  Prince of Wales-Hyder   
33095          NaN          38      0.2         2198  Prince of Wales-Hyder   
33096          NaN        2484      0.4         2275               Wrangell   

                                  county_weights             county_names_all  \
0                   {'72001':99.43,'72141':0.57}              Adjuntas|Utuado   
1                                  {'72003':100}                       Aguada   
2                                  {'72005':100}                    Aguadilla   
3      {'72093':94.88,'72121':1.35,'72153':3.78}  Maricao|Yauco|Sabana Grande   
4                   {'72003':0.55,'72011':99.45}                Añasco|Aguada   
...                                          ...                          ...   
33092                              {'02198':100}        Prince of Wales-Hyder   
33093                              {'02198':100}        Prince of Wales-Hyder   
33094                              {'02198':100}        Prince of Wales-Hyder   
33095                              {'02198':100}        Prince of Wales-Hyder   
33096                              {'02275':100}                     Wrangell   

         county_fips_all  imprecise  military             timezone  
0            72001|72141      False     False  America/Puerto_Rico  
1                  72003      False     False  America/Puerto_Rico  
2                  72005      False     False  America/Puerto_Rico  
3      72093|72153|72121      False     False  America/Puerto_Rico  
4            72011|72003      False     False  America/Puerto_Rico  
...                  ...        ...       ...                  ...  
33092              02198      False     False        America/Sitka  
33093              02198      False     False        America/Sitka  
33094              02198      False     False   America/Metlakatla  
33095              02198      False     False        America/Sitka  
33096              02275      False     False        America/Sitka  

[33097 rows x 18 columns]

# Clean Data

In [13]:
#Convert data to dataframe, perform some basic operations
#Keep zip, lat, lng, city, state_name, county_fips, county_name, county_fips_all
#drop the rest

cleanUSzip_df = pd.DataFrame(dataMain)

cleanUSzip_df = cleanUSzip_df.rename(columns={"lng":"long","state_name":"state"})
cleanUSzip_df.drop(columns=['zcta','parent_zcta','county_name', 'density', 'county_weights', 'county_names_all', 'imprecise', 'military', 'timezone', 'county_fips_all'],inplace=True)
cleanUSzip_df.dropna(how="any", inplace=True)
#made sure city and state_id were string before mergeing both columns
cleanUSzip_df['city'] = cleanUSzip_df.city.astype(str)
cleanUSzip_df['state_id'] = cleanUSzip_df.state_id.astype(str)
cleanUSzip_df['cityST'] = cleanUSzip_df[['city', 'state_id']].agg(', '.join, axis=1)
cleanUSzip_df['lat'] = cleanUSzip_df.lat.astype(str)
cleanUSzip_df['long'] = cleanUSzip_df.long.astype(str)
cleanUSzip_df['us_Lat_Long'] = cleanUSzip_df[['lat', 'long']].agg(','.join, axis=1)


cleanUSzip_df

zip                 lat                 long         city state_id  \
0        601  18.180039999999998   -66.75218000000001     Adjuntas       PR   
1        602            18.36073            -67.17517       Aguada       PR   
2        603            18.45439   -67.12201999999999    Aguadilla       PR   
3        606            18.16724            -66.93828      Maricao       PR   
4        610            18.29032   -67.12243000000001       Anasco       PR   
...      ...                 ...                  ...          ...      ...   
33092  99923  55.977959999999996           -130.03671        Hyder       AK   
33093  99925  55.557669999999995           -132.97627      Klawock       AK   
33094  99926  55.126169999999995           -131.48928   Metlakatla       AK   
33095  99927  56.251000000000005           -133.37571  Point Baker       AK   
33096  99929             56.3695  -131.93648000000002     Wrangell       AK   

             state  population  county_fips           cityST  \
0      Puerto Rico       17242        72001     Adjuntas, PR   
1      Puerto Rico       38442        72003       Aguada, PR   
2      Puerto Rico       48814        72005    Aguadilla, PR   
3      Puerto Rico        6437        72093      Maricao, PR   
4      Puerto Rico       27073        72011       Anasco, PR   
...            ...         ...          ...              ...   
33092       Alaska          15         2198        Hyder, AK   
33093       Alaska         927         2198      Klawock, AK   
33094       Alaska        1635         2198   Metlakatla, AK   
33095       Alaska          38         2198  Point Baker, AK   
33096       Alaska        2484         2275     Wrangell, AK   

                                 us_Lat_Long  
0      18.180039999999998,-66.75218000000001  
1                         18.36073,-67.17517  
2                18.45439,-67.12201999999999  
3                         18.16724,-66.93828  
4                18.29032,-67.12243000000001  
...                                      ...  
33092          55.977959999999996,-130.03671  
33093          55.557669999999995,-132.97627  
33094          55.126169999999995,-131.48928  
33095          56.251000000000005,-133.37571  
33096            56.3695,-131.93648000000002  

[33097 rows x 10 columns]

# Save as CSV

In [14]:
#Save as a csv
#Using encoding="utf-8" to avoid issues later
cleanUSzip_df.to_csv("Resources/zip_Lat_Lon_US.csv", encoding="utf-8", index=False)